In [3]:
!pip install pyaudio
!pip install wave

In [4]:
import pyaudio
import wave

def record_audio(filename, duration=5):
    chunk = 1024
    format = pyaudio.paInt16
    channels = 1
    rate = 44100

    p = pyaudio.PyAudio()

    stream = p.open(format=format,
                    channels=channels,
                    rate=rate,
                    input=True,
                    frames_per_buffer=chunk)

    frames = []

    for _ in range(int(rate / chunk * duration)):
        data = stream.read(chunk)
        frames.append(data)

    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(filename, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(format))
    wf.setframerate(rate)
    wf.writeframes(b''.join(frames))
    wf.close()


In [5]:
!pip install librosa
!pip install numpy

  Using cached librosa-0.10.2.post1-py3-none-any.whl.metadata (8.6 kB)
  Using cached audioread-3.0.1-py3-none-any.whl.metadata (8.4 kB)
  Using cached soundfile-0.12.1-py2.py3-none-win_amd64.whl.metadata (14 kB)
  Using cached pooch-1.8.2-py3-none-any.whl.metadata (10 kB)
  Using cached soxr-0.3.7-cp311-cp311-win_amd64.whl.metadata (5.7 kB)
Using cached librosa-0.10.2.post1-py3-none-any.whl (260 kB)
Using cached audioread-3.0.1-py3-none-any.whl (23 kB)
Using cached pooch-1.8.2-py3-none-any.whl (64 kB)
Using cached soundfile-0.12.1-py2.py3-none-win_amd64.whl (1.0 MB)
Using cached soxr-0.3.7-cp311-cp311-win_amd64.whl (184 kB)


In [6]:
import librosa
import numpy as np

def extract_features(filename):
    y, sr = librosa.load(filename, sr=None)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    return np.mean(mfccs.T, axis=0)


In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

def create_model(input_shape):
    model = Sequential()
    model.add(LSTM(128, input_shape=input_shape, return_sequences=True))
    model.add(LSTM(128))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(len(vocabulary), activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [8]:
import tkinter as tk
from tkinter import messagebox

def start_recording():
    record_audio('output.wav', duration=10)
    transcription = transcribe_audio('output.wav')
    result_label.config(text=transcription)

def stop_recording():
    # Implement stop recording logic if required
    messagebox.showinfo("Info", "Recording stopped.")

root = tk.Tk()
root.title("STT Transcription System")

record_button = tk.Button(root, text="Record", command=start_recording)
record_button.pack()

stop_button = tk.Button(root, text="Stop", command=stop_recording)
stop_button.pack()

result_label = tk.Label(root, text="")
result_label.pack()

root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "D:\anaconda\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\merlin cj\AppData\Local\Temp\ipykernel_8856\2138148140.py", line 6, in start_recording
    transcription = transcribe_audio('output.wav')
                    ^^^^^^^^^^^^^^^^
NameError: name 'transcribe_audio' is not defined
Exception in Tkinter callback
Traceback (most recent call last):
  File "D:\anaconda\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\merlin cj\AppData\Local\Temp\ipykernel_8856\2138148140.py", line 6, in start_recording
    transcription = transcribe_audio('output.wav')
                    ^^^^^^^^^^^^^^^^
NameError: name 'transcribe_audio' is not defined
